In [ ]:
import pandas as pd
import sqlite3

In [ ]:
df = pd.read_csv('Customer_support_data.csv')

In [ ]:
df.head()

In [ ]:
# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
df.to_sql('customer_support', conn, index=False, if_exists='replace')

# Confirm table created
pd.read_sql("SELECT * FROM customer_support LIMIT 5", conn)

In [ ]:
# 1. All 'Order Related' issues
q1 = "SELECT * FROM customer_support WHERE category = 'Order Related'"
pd.read_sql(q1, conn)

In [ ]:
# 2. Top 5 most frequent Sub-categories
q2 = """
SELECT "Sub-category", COUNT(*) AS issue_count
FROM customer_support
GROUP BY "Sub-category"
ORDER BY issue_count DESC
LIMIT 5
"""
pd.read_sql(q2, conn)

In [ ]:
# Create dummy agent_details table
agent_details = pd.DataFrame({
    'Agent_name': df['Agent_name'].dropna().unique()[:5],
    'Experience': [1, 3, 5, 2, 4]
})
agent_details.to_sql('agent_details', conn, index=False, if_exists='replace')

# 3. Join to add experience
q3 = """
SELECT cs.*, ad.Experience
FROM customer_support cs
JOIN agent_details ad ON cs.Agent_name = ad.Agent_name
"""
pd.read_sql(q3, conn)

In [ ]:
# 4. Agents who handled more than average number of issues
q4 = """
SELECT Agent_name, COUNT(*) AS total_issues
FROM customer_support
GROUP BY Agent_name
HAVING COUNT(*) > (
    SELECT AVG(issue_count) FROM (
        SELECT Agent_name, COUNT(*) AS issue_count
        FROM customer_support
        GROUP BY Agent_name
    )
)
"""
pd.read_sql(q4, conn)

In [ ]:
# 5. Avg handling time per category
q5 = """
SELECT category, AVG(CAST(connected_handling_time AS FLOAT)) AS avg_handling_time
FROM customer_support
GROUP BY category
"""
pd.read_sql(q5, conn)

In [ ]:
# 6. Total item price per product category
q6 = """
SELECT Product_category, SUM(CAST(Item_price AS FLOAT)) AS total_revenue
FROM customer_support
GROUP BY Product_category
"""
pd.read_sql(q6, conn)

In [ ]:
# 7. Create view for Inbound performance
q7 = """
CREATE VIEW inbound_summary AS
SELECT category, "Sub-category", COUNT(*) AS total_issues,
       AVG(CAST("CSAT Score" AS FLOAT)) AS avg_CSAT
FROM customer_support
WHERE channel_name = 'Inbound'
GROUP BY category, "Sub-category"
"""
conn.execute(q7)

In [ ]:
pd.read_sql("SELECT * FROM inbound_summary", conn)

In [ ]:
# 8. Add indexes for performance
conn.execute("CREATE INDEX IF NOT EXISTS idx_category ON customer_support(category)")
conn.execute("CREATE INDEX IF NOT EXISTS idx_agent_name ON customer_support(Agent_name)")
conn.execute("CREATE INDEX IF NOT EXISTS idx_channel_category ON customer_support(channel_name, category)")


In [ ]:
#Top-performing agents by average CSAT score
q1 = """
SELECT Agent_name, AVG(CAST("CSAT Score" AS FLOAT)) AS avg_score
FROM customer_support
GROUP BY Agent_name
ORDER BY avg_score DESC
LIMIT 5;
"""
top_agents = pd.read_sql(q1, conn)
top_agents

In [ ]:
#Average handling time by Sub-category and Agent
q2 = """
SELECT "Sub-category", Agent_name, AVG(CAST(connected_handling_time AS FLOAT)) AS avg_time
FROM customer_support
GROUP BY "Sub-category", Agent_name
ORDER BY avg_time ASC;
"""
avg_time = pd.read_sql(q2, conn)
avg_time

In [ ]:
#Category-wise resolution effectiveness using CSAT score
q3 = """
SELECT category, COUNT(*) AS total, AVG(CAST("CSAT Score" AS FLOAT)) AS avg_CSAT
FROM customer_support
GROUP BY category
ORDER BY avg_CSAT DESC;
"""
category_effectiveness = pd.read_sql(q3, conn)
category_effectiveness

In [ ]:
#Using CASE: Categorizing CSAT Levels
q4 = """
SELECT
  Agent_name,
  CASE
    WHEN "CSAT Score" >= 4 THEN 'High'
    WHEN "CSAT Score" >= 2 THEN 'Medium'
    ELSE 'Low'
  END AS CSAT_Level,
  COUNT(*) AS issue_count
FROM customer_support
GROUP BY Agent_name, CSAT_Level;
"""
csat_levels = pd.read_sql(q4, conn)
csat_levels